# Requierments


In [1]:
!pip install findspark
!pip install confluent-kafka
# Downloaded from https://mvnrepository.com/artifact/org.apache.spark/spark-streaming-kafka-0-8-assembly_2.11
!wget https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.0/spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar

--2020-04-13 20:37:14--  https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.0/spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.112.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.112.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13385346 (13M) [application/java-archive]
Saving to: ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar.1’

spark-streaming-kaf 100%[===================>]  12.76M  4.78MB/s    in 2.7s    

2020-04-13 20:37:17 (4.78 MB/s) - ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar.1’ saved [13385346/13385346]



In [2]:
import os
import findspark
findspark.init('/usr/local/spark/spark-2.4.0-bin-hadoop2.7')
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar pyspark-shell'

---

In [3]:
from pyspark import SparkConf, SparkContext
# from operator import add
# import sys
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import pickle
from pyspark.mllib.feature import HashingTF, IDF, StandardScaler
from pyspark.mllib.classification import LogisticRegressionModel
from pyspark.mllib.linalg import Vectors

APP_NAME = "BigData"
sc = SparkContext(appName=APP_NAME)
ssc = StreamingContext(sc, 2)
sc

<SparkContext master=local[*] appName=BigData>

In [ ]:
with open('./data/hashedTF.pickle', 'rb') as f:
    hashingTF = pickle.load(f)
    
htfVectors = sc.textFile("./data/htfVectors").map(Vectors.parse)
idf = IDF().fit(htfVectors)

tfidf =  idf.transform(htfVectors)

scaler = StandardScaler().fit(tfidf)

model = LogisticRegressionModel.load(sc, "./models/Logistic_Regression")

In [ ]:
topic = "test"
brokers = "localhost:9092"
kvs = KafkaUtils.createDirectStream(ssc, [topic], {"metadata.broker.list": brokers})


In [ ]:
def to_ngram(payload_obj):
    n=2
    payload = str(payload_obj)
    ngrams = []
    for i in range(0,len(payload)-n+1):
        ngrams.append(payload[i:i+n])
    return ngrams


def get_prediction(line):
    try:
        
        predictions = model.predict(scaler.transform(idf.transform(hashingTF.transform(to_ngram(line.collect()[0])))))
        print(line.collect() + predictions.collect())
    except : 
        print('---------------No data----------------')

def print_line(line):
    print(line.collect())

In [ ]:
#lines = ssc.socketTextStream("localhost", 9092)
#ngrams = kvs.flatMap(lambda x : to_ngram(x))
kvs.foreachRDD(get_prediction)
#kvs.foreachRDD(to_ngram)
ssc.start()
ssc.awaitTermination()  

---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data----------------
---------------No data---

In [ ]:
KafkaUtils.createDirectStream?